# Análisis Predictivo de Datos
## Proyecto de aula: Análisis del IPC en Colombia


### Integrantes

- Alejandro Córdoba Ríos
- Juan Camilo Manjarrés Baena
- Adrián David Perdomo Echeverri


### Descripción

Este proyecto de aula busca hacer un análisis histórico de los datos proporcionados por el **Departamento Administrativo Nacional de Estadísticas (DANE)** con respecto al **Índice de Precios al Consumidor (IPC)**, con el fin de predecir la variación de este indicador económico en el futuro. Se obtienen datos de la variación mensual del IPC total, por divisiones de bienes y servicios y categorizados por ciudades.

Se toman datos de los últimos 10 años (120 meses), desde septiembre de 2014 hasta agosto de 2024.

Las divisiones de bienes y servicios que identifica el DANE son:
- Total (IPC general sin categorizar)
- Alimentos y bebidas no alcohólicas
- Bebidas alcohólicas y tabaco
- Prendas de vestir y calzado
- Alojamiento, agua, electricidad, gas y otros combustibles
- Muebles, artículos para el hogar y para la conservación ordinaria del hogar
- Salud
- Transporte
- Información y comunicación
- Recreación y cultura
- Educación
- Restaurantes y hoteles
- Bienes y servicios diversos

*NOTA: Las categorías varían según los años. Para el conjunto de datos de 5 años atrás y 10 años atrás, la cantidad y nombres de las categorías varían significativamente.*


Y las ciudades listadas son:
- Nacional (IPC total en Colombia)
- Medellín
- Barranquilla
- Bogotá, D.C.
- Cartagena De Indias
- Tunja
- Manizales
- Florencia
- Popayán
- Valledupar
- Montería
- Neiva
- Riohacha
- Santa Marta
- Villavicencio
- Pasto
- Cúcuta
- Armenia
- Pereira
- Bucaramanga
- Sincelejo
- Ibagué
- Cali
- Otras Areas Urbanas

*NOTA: La cantidad y ciudades listadas también cambian en algunos períodos. Sin embargo, las ciudades principales se encuentran en todos los períodos.*

Los datos se obtienen del sitio oficial del DANE: https://www.dane.gov.co/index.php/comunicados-y-boletines/indice-de-precios-y-costos/ipc

### Recolección de los datos

Se obtienen los anexos de los reportes históricos mensuales que se encuentran en el enlace citado anteriormente, los cuales son documentos con formato *.xlsx* (*.xls* para los reportes más antiguos). Estos archivos de Excel contienen varias hojas con información ordenada y categorizada de distintas maneras, pero para el objetivo del proyecto de aula, nos concentraremos en los datos de la **hoja "4"**, que corresponde a **"Variación mensual, total y por divisiones de bienes y servicios, según ciudades"**.

Como son en total 120 archivos (un archivo por cada mes, desde septiembre de 2014 hasta agosto de 2024) y lo ideal es tener todos los datos en un solo archivo, vamos a iterar cada archivo para obtener su información e irla recolectando y ordenando en un archivo final con formato *.csv*. Se renombraron los archivos para seguir una convención específica, que nos servirá para que en la iteración se identifique a qué mes y año corresponde la información. Esta convención es *año-mes.xlsx*, donde año es el número con 4 dígitos y mes es el número con 2 dígitos (por ejemplo: *2023-07.xlsx*, que corresponde a julio del 2023).

Al final esperamos tener un único archivo con formato *.csv* donde cada fila contenga la siguiente información:
- Ciudad [texto]
- Categoría (división de bienes y servicios) [texto]
- Año [número]
- Mes [número]
- IPC [número]

In [ ]:
%pip install pandas
%pip install openpyxl
%pip install xlrd

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ------------- -------------------------- 81.9/250.9 kB 2.3 MB/s eta 0:00:01
   -------------------------------- ------- 204.8/250.9 kB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#Carga de datos desde local

In [25]:
import os
import pandas as pd

data_folder = 'data'
final_data = []
for filename in os.listdir(data_folder):
    print(f'Processing {filename}...')
    slug, _ = os.path.splitext(filename)
    year, month = map(int, slug.split('-'))

    if year >= 2019:
        sheet_name, header, nrows, usecols = '4', 6, 24, 'A:N'
    elif year == 2018 or (year == 2017 and month == 12):
        sheet_name, header, nrows, usecols = 'Anexo4', 7, 25, 'A:K'
    elif year == 2017 and 6 <= month <= 11:
        sheet_name, header, nrows, usecols = 'Anexo4', 5, 25, 'A:K'
    else:
        sheet_name, header, nrows, usecols = 'Anexo4', 3, 25, 'A:K'        

    data = pd.read_excel(os.path.join(data_folder, filename), sheet_name=sheet_name, header=header, usecols=usecols, nrows=nrows)

    melted_data = data.melt(id_vars=['Ciudades'], var_name='Categoría', value_name='IPC')
    melted_data['Ciudades'] = melted_data['Ciudades'].astype(str)
    melted_data['Categoría'] = melted_data['Categoría'].astype(str)
    melted_data['Año'] = int(year)
    melted_data['Mes'] = int(month)
    melted_data['IPC'] = melted_data['IPC'].astype(float)
    melted_data.rename(columns={'Ciudades': 'city', 'Categoría': 'category', 'Año': 'year', 'Mes': 'month', 'IPC': 'ipc'}, inplace=True)
    melted_data['city'] = melted_data['city'].replace({'Bogotá D.C.': 'Bogotá', 'Bogotá, D.C.': 'Bogotá'})
    melted_data['city'] = melted_data['city'].replace({'Otras Areas Urbanas': 'Otras áreas urbanas'})
    melted_data['city'] = melted_data['city'].replace({'Total IPC': 'Nacional', 'Total Nacional': 'Nacional'})

    final_data.append(melted_data)

final_df = pd.concat(final_data, ignore_index=True)
final_name = 'ipc_from_2014-09_to_2024-08.csv'
final_df.to_csv(final_name, index=False)
print(f'Final data saved to {final_name}')

Processing 2014-09.xls...
Processing 2014-10.xls...
Processing 2014-11.xls...
Processing 2014-12.xls...
Processing 2015-01.xls...
Processing 2015-02.xls...
Processing 2015-03.xls...
Processing 2015-04.xls...
Processing 2015-05.xls...
Processing 2015-06.xls...
Processing 2015-07.xls...
Processing 2015-08.xls...
Processing 2015-09.xls...
Processing 2015-10.xls...
Processing 2015-11.xls...
Processing 2015-12.xls...
Processing 2016-01.xls...
Processing 2016-02.xls...
Processing 2016-03.xls...
Processing 2016-04.xls...
Processing 2016-05.xls...
Processing 2016-06.xls...
Processing 2016-07.xls...
Processing 2016-08.xls...
Processing 2016-09.xls...
Processing 2016-10.xls...
Processing 2016-11.xls...
Processing 2016-12.xls...
Processing 2017-01.xls...
Processing 2017-02.xls...
Processing 2017-03.xls...
Processing 2017-04.xls...
Processing 2017-05.xls...
Processing 2017-06.xls...
Processing 2017-07.xls...
Processing 2017-08.xls...
Processing 2017-09.xls...
Processing 2017-10.xls...
Processing 2

In [27]:
df = pd.read_csv("./ipc_from_2014-09_to_2024-08.csv")

df['category'].unique()

array(['Alimentos', 'Vivienda', 'Vestuario', 'Salud', 'Educación',
       'Diversión', 'Transporte', 'Comunicaciones', 'Otros gastos',
       'Total', 'Alimentos Y Bebidas No Alcohólicas',
       'Bebidas Alcohólicas Y Tabaco', 'Prendas De Vestir Y Calzado',
       'Alojamiento, Agua, Electricidad, Gas Y Otros Combustibles',
       'Muebles, Artículos Para El Hogar Y Para La Conservación Ordinaria Del Hogar',
       'Información Y Comunicación', 'Recreación Y Cultura',
       'Restaurantes Y Hoteles', 'Bienes Y Servicios Diversos'],
      dtype=object)

#Carga de datos desde kaggle

https://www.kaggle.com/datasets/alejandrocrdobaros/ipc-from-september-2022-to-august-2024-by-dane/


In [ ]:
import pandas as pd
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alejandrocrdobaros/ipc-from-september-2022-to-august-2024-by-dane",path='ipc_from_2022-09_to_2024-08.csv')

df = pd.read_csv(path)
df.head()

,city,category,ipc,year,month
0,Medellín,Alimentos Y Bebidas No Alcohólicas,1.90,2022,9
1,Barranquilla,Alimentos Y Bebidas No Alcohólicas,0.77,2022,9
2,"Bogotá, D.C.",Alimentos Y Bebidas No Alcohólicas,1.83,2022,9
3,Cartagena De Indias,Alimentos Y Bebidas No Alcohólicas,1.36,2022,9
4,Tunja,Alimentos Y Bebidas No Alcohólicas,1.84,2022,9


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7176 entries, 0 to 7175
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   city      7176 non-null   object 
 1   category  7176 non-null   object 
 2   ipc       7176 non-null   float64
 3   year      7176 non-null   int64  
 4   month     7176 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 280.4+ KB


In [ ]:
df[['year', 'month']] = df[['year', 'month']].astype("object")

df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7176 entries, 2022-09-01 to 2024-08-01
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   city      7176 non-null   object 
 1   category  7176 non-null   object 
 2   ipc       7176 non-null   float64
 3   year      7176 non-null   object 
 4   month     7176 non-null   object 
dtypes: float64(1), object(4)
memory usage: 336.4+ KB


array([2022, 2023, 2024], dtype=object)

In [ ]:
df.head()

,city,category,ipc,year,month
0,Medellín,Alimentos Y Bebidas No Alcohólicas,1.90,2022,9
1,Barranquilla,Alimentos Y Bebidas No Alcohólicas,0.77,2022,9
2,"Bogotá, D.C.",Alimentos Y Bebidas No Alcohólicas,1.83,2022,9
3,Cartagena De Indias,Alimentos Y Bebidas No Alcohólicas,1.36,2022,9
4,Tunja,Alimentos Y Bebidas No Alcohólicas,1.84,2022,9


#Crear index de tiempo y separar las variables


In [ ]:
from sklearn.model_selection import train_test_split
import datetime

df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))

df.set_index('date', inplace=True)

df.dropna(inplace=True)

X = df.drop(columns=['ipc'])
y = df['ipc']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size=0.7)



#Preprocesar las variables


In [ ]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
ohe = OneHotEncoder(sparse_output=False, drop='first')
oe_month = OrdinalEncoder()
oe_year = OrdinalEncoder()


preprocessor = ColumnTransformer(transformers=[
    ('ohe', ohe, ['city',"category"]),  # OneHot para  city y category
    ('oe_year', oe_year, ['year']),  # Ordinal para year
    ('oe_month', oe_month, ['month'])  # Ordinal para month
], remainder='passthrough')

#Entrenar el modelo

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([('preprocessor', preprocessor), ('model', Ridge())]) # Se define un pipeline con el preprocesador y el modelo Ridge
alpha = np.logspace(-4, 2)
alpha = np.logspace(-4, 2)
grid = {'model__alpha':alpha} # Se define la grilla de hiperparámetros a sintonizar
grid_search = GridSearchCV(estimator=pipe, # Se define el modelo a sintonizar, que incluye el preprocesador y el modelo Ridge
                          param_grid=grid, # Se define la grilla de hiperparámetros que se va a sintonizar
                          cv=5, # Se define el número de folds en la validación cruzada
                          scoring='neg_root_mean_squared_error') # Se define la métrica de evaluación, en este caso RMSE
grid_search.fit(X_train, y_train)

print(f'Mejor RMSE obtenido fue {-grid_search.best_score_:.3} con hiperparámetros de {grid_search.best_params_}.')
print(f'Error de prueba: {np.sqrt(mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test))):.3f}')


Mejor RMSE obtenido fue 0.962 con hiperparámetros de {'model__alpha': 42.91934260128778}.
Error de prueba: 1.050


In [ ]:
pesos = pd.DataFrame(data=grid_search.best_estimator_.named_steps['model'].coef_, # Se extraen los coeficientes del modelo Ridge
                     index=grid_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out().tolist(), # Se extraen los nombres de las variables
                     columns=['Coeficiente'])
pesos.loc['intercepto'] = grid_search.best_estimator_.named_steps['model'].intercept_ # Se extrae el intercepto del modelo Ridge
pesos

,Coeficiente
ohe__city_Barranquilla,-0.037503
"ohe__city_Bogotá, D.C.",0.005447
ohe__city_Bucaramanga,-0.027305
ohe__city_Cali,-0.051254
ohe__city_Cartagena De Indias,0.031887
ohe__city_Cúcuta,0.091097
ohe__city_Florencia,0.068062
ohe__city_Ibagué,0.011226
ohe__city_Manizales,-0.035178
ohe__city_Medellín,0.038373


In [ ]:
df.describe()

,ipc
count,7176.000000
mean,0.647435
std,1.083605
min,-7.290000
25%,0.070000
50%,0.460000
75%,0.950000
max,13.570000
